In [1]:
import json
import requests
from bs4 import BeautifulSoup
import urllib.parse
import os
import re
import time

# Tải dữ liệu tác phẩm

In [2]:
current_folder = os.getcwd()
data_folder_path = os.path.abspath(os.path.join(current_folder, '..', 'Data'))

# Đường dẫn file đầu vào và đầu ra
input_file_path = os.path.join(data_folder_path, 'link_href.csv')
output_file_path = os.path.join(data_folder_path, 'wiki_data.json')

# Header giả lập trình duyệt
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

print(f" Thư mục dữ liệu: {data_folder_path}")
print(f" File input: {input_file_path}")
print(f" File output: {output_file_path}")


 Thư mục dữ liệu: d:\STUDY DOCUMENT\4th YEAR\NLP\Vietnamese_literature\Data
 File input: d:\STUDY DOCUMENT\4th YEAR\NLP\Vietnamese_literature\Data\link_href.csv
 File output: d:\STUDY DOCUMENT\4th YEAR\NLP\Vietnamese_literature\Data\wiki_data.json


In [3]:

def clean_text(text):
    """Hàm làm sạch sơ bộ văn bản"""
    # Xóa khoảng trắng thừa (ví dụ: "  " -> " ")
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def get_wiki_content_optimized(url):
    try:
        # Giải mã URL
        decoded_url = urllib.parse.unquote(url)
        
        response = requests.get(decoded_url, headers=headers, timeout=15)
        if response.status_code != 200:
            return None 

        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 1. Tìm vùng nội dung chính
        content_div = soup.find('div', {'id': 'mw-content-text'})
        if not content_div:
            return ""

        # 2. LÀM SẠCH HTML
        # Xóa số mũ tham khảo [1], [2]...
        for tag in content_div.find_all('sup', class_='reference'):
            tag.decompose()
            
        # Xóa bảng thông tin, điều hướng, mục lục
        for tag in content_div.find_all(['table', 'div'], class_=['infobox', 'navbox', 'toc', 'mw-empty-elt']):
            tag.decompose()
            
        # Xóa các nút "sửa"
        for tag in content_div.find_all('span', class_='mw-editsection'):
            tag.decompose()

        # 3. TRÍCH XUẤT NỘI DUNG
        parser_output = content_div.find('div', class_='mw-parser-output')
        extracted_text = []
        
        if parser_output:
            stop_keywords = ['tham khảo', 'chú thích', 'liên kết ngoài', 'xem thêm', 'đọc thêm']
            
            for element in parser_output.children:
                if element.name is None: continue

                # Kiểm tra điểm dừng
                if element.name in ['h2', 'h3']:
                    header_text = element.get_text().lower()
                    if any(kw in header_text for kw in stop_keywords):
                        break 
                
                # Lấy nội dung
                if element.name in ['p', 'h2', 'h3', 'h4', 'ul', 'ol', 'dl']:
                    text = element.get_text(separator=' ').strip()
                    clean_t = clean_text(text)
                    if clean_t:
                        extracted_text.append(clean_t)

        return "\n\n".join(extracted_text)

    except Exception as e:
        print(f"Lỗi ngoại lệ khi crawl {url}: {e}")
        return ""




In [4]:
final_data = []
LINK_LIMIT = 57  

if os.path.exists(input_file_path):
    print("\n Đang đọc file CSV và bắt đầu xử lý...")
    
    with open(input_file_path, 'r', encoding='utf-8-sig') as f:
        lines = f.readlines()
    
    print(f" Tìm thấy tổng cộng {len(lines)} dòng trong file.")
    
    processed_count = 0 # Biến đếm số link đã xử lý

    for i, line in enumerate(lines):
        line = line.strip()
        if not line: continue
        
        # Xử lý tách cột
        parts = line.split(',')
        if len(parts) < 2:
            parts = line.split(';')
            
        if len(parts) >= 2:
            name = parts[0].strip().replace('"', '')
            url = parts[-1].strip().replace('"', '')
            
            # Bỏ qua tiêu đề hoặc link hỏng
            if "http" not in url:
                continue

            if processed_count >= LINK_LIMIT:
                print(f"\n Đã đạt giới hạn {LINK_LIMIT} link đầu tiên. Dừng xử lý các link còn lại.")
                break

            print(f"[{processed_count + 1}/{LINK_LIMIT}] ⏳ Đang tải: {name}...", end=" ")
            
            content = get_wiki_content_optimized(url)
            
            if content:
                final_data.append({
                    "ten_tac_gia": name,
                    "link": url,
                    "noi_dung": content
                })
                print(f" Thành công ({len(content)} ký tự)")
            else:
                print(" Thất bại hoặc không có nội dung")
            
            processed_count += 1
                
    if final_data:
        print(f"\nĐang lưu {len(final_data)} mục vào: {output_file_path}")
        with open(output_file_path, 'w', encoding='utf-8') as f:
            json.dump(final_data, f, ensure_ascii=False, indent=4)
        print("HOÀN TẤT!")
    else:
        print(" Không lấy được dữ liệu nào.")

else:
    print(f" LỖI: Không tìm thấy file đầu vào tại: {input_file_path}")


 Đang đọc file CSV và bắt đầu xử lý...
 Tìm thấy tổng cộng 110 dòng trong file.
[1/57] ⏳ Đang tải: Ai tư vãn...  Thành công (560 ký tự)
[2/57] ⏳ Đang tải: Bác ơi!...  Thành công (1346 ký tự)
[3/57] ⏳ Đang tải: Bài thơ về tiểu đội xe không kính...  Thành công (647 ký tự)
[4/57] ⏳ Đang tải: Bình Ngô đại cáo...  Thành công (1466 ký tự)
[5/57] ⏳ Đang tải: Bông hồng cài áo...  Thành công (1984 ký tự)
[6/57] ⏳ Đang tải: Cái mặt không chơi được...  Thành công (879 ký tự)
[7/57] ⏳ Đang tải: Câu chuyện đối đáp của người tiều phu ở núi Na...  Thành công (646 ký tự)
[8/57] ⏳ Đang tải: Châu bản triều Nguyễn...  Thành công (5400 ký tự)
[9/57] ⏳ Đang tải: Chí Phèo...  Thành công (306 ký tự)
[10/57] ⏳ Đang tải: Chiếc thuyền ngoài xa...  Thành công (575 ký tự)
[11/57] ⏳ Đang tải: Chiếu thư đánh Chiêm...  Thành công (208 ký tự)
[12/57] ⏳ Đang tải: Chinh phụ ngâm...  Thành công (446 ký tự)
[13/57] ⏳ Đang tải: Chuyện người con gái Nam Xương...  Thành công (343 ký tự)
[14/57] ⏳ Đang tải: Công dư tiệp ký.

In [5]:
def clean_sentence(text):
    """
    Hàm làm sạch một câu văn:
    - Xóa ký tự xuống dòng
    - Xóa số tham chiếu [1] nếu còn sót
    - Xóa các ký tự đặc biệt rác (giữ lại tiếng Việt và dấu câu chuẩn)
    - Chuẩn hóa khoảng trắng
    """
    # 1. Xóa xuống dòng
    text = text.replace('\n', ' ').replace('\r', ' ')
    
    # 2. Xóa số mũ tham chiếu dạng [1], [12] (đề phòng còn sót)
    text = re.sub(r'\[\d+\]', '', text)
    
    # 3. Giữ lại: Chữ cái (bao gồm tiếng Việt), số, khoảng trắng, và các dấu câu cơ bản (. , ? ! ; : " - ( ))
    # Loại bỏ các ký tự lạ như @ # $ % ^ & * ...
    text = re.sub(r'[^\w\s\.,\?!\;:"\-\(\)\u00C0-\u1EF9]', ' ', text)
    
    # 4. Xóa khoảng trắng thừa (ví dụ "  " -> " ")
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def split_into_sentences(text):
    """
    Tách đoạn văn thành danh sách các câu dựa vào dấu chấm, hỏi, cảm thán.
    Sử dụng Lookbehind (?<=...) để giữ lại dấu câu ở cuối câu.
    """
    # Tách khi gặp . ! ? theo sau là khoảng trắng hoặc hết dòng
    sentences = re.split(r'(?<=[.!?])\s+', text)
    return sentences

In [ ]:
input_file_path = os.path.join(data_folder_path, 'wiki_data.json')
output_file_path = os.path.join(data_folder_path, 'wiki_sentences_1.json')

In [7]:
processed_data = []

if os.path.exists(input_file_path):
    print("\n Đang đọc file JSON và xử lý tách câu...")
    
    try:
        with open(input_file_path, 'r', encoding='utf-8') as f:
            raw_data = json.load(f)
            
        print(f" Tìm thấy {len(raw_data)} bài viết gốc.")
        
        total_sentences = 0
        
        for entry in raw_data:
            author_name = entry.get('ten_tac_gia', 'Unknown')
            full_content = entry.get('noi_dung', '')
            
            # Bước 1: Tách thành các câu lẻ
            sentences_list = split_into_sentences(full_content)
            
            for sent in sentences_list:
                # Bước 2: Làm sạch câu
                cleaned_sent = clean_sentence(sent)
                
                # Bước 3: Lọc rác (Chỉ lấy câu có độ dài > 10 ký tự và bắt đầu bằng chữ hoa)
                # Điều kiện len > 10 giúp loại bỏ các tiêu đề ngắn hoặc vụn vặt
                if len(cleaned_sent) > 10:
                    processed_data.append({
                        "ten_tac_gia": author_name,
                        "cau_van": cleaned_sent
                    })
                    total_sentences += 1

        # Lưu kết quả
        print(f"\n Đã tách và làm sạch được: {total_sentences} câu.")
        print(f" Đang ghi file...")
        
        with open(output_file_path, 'w', encoding='utf-8') as f:
            json.dump(processed_data, f, ensure_ascii=False, indent=4)
            
        print(" HOÀN TẤT! File json mới đã sẵn sàng.")
        
        # In thử 5 câu đầu tiên để kiểm tra
        if processed_data:
            print("\n--- [XEM THỬ 5 CÂU ĐẦU TIÊN] ---")
            print(json.dumps(processed_data[:5], ensure_ascii=False, indent=4))

    except Exception as e:
        print(f" Lỗi khi xử lý: {e}")



 Đang đọc file JSON và xử lý tách câu...
 Tìm thấy 54 bài viết gốc.

 Đã tách và làm sạch được: 386 câu.
 Đang ghi file...
 HOÀN TẤT! File json mới đã sẵn sàng.

--- [XEM THỬ 5 CÂU ĐẦU TIÊN] ---
[
    {
        "ten_tac_gia": "Ai tư vãn",
        "cau_van": "Ai tư vãn ( chữ Hán : 哀思挽) là một tác phẩm trong văn chương Việt Nam , viết bằng chữ Nôm ."
    },
    {
        "ten_tac_gia": "Ai tư vãn",
        "cau_van": "Người ta tương truyền bài thơ này là do Bắc Cung Hoàng hậu Lê Ngọc Hân viết khóc phu quân là Quang Trung Hoàng đế Nguyễn Huệ ."
    },
    {
        "ten_tac_gia": "Ai tư vãn",
        "cau_van": "Theo ý kiến của Thuần Phong đã nói trong sách \"Chinh phụ ngâm khúc giảng luận\" (do Á Châu xuất bản), bài thơ này có chịu ảnh hưởng bản dịch Chinh phụ ngâm của bà Đoàn Thị Điểm ."
    },
    {
        "ten_tac_gia": "Ai tư vãn",
        "cau_van": "Đây là một tác phẩm văn vần viết theo thể \"ngâm\" , tức song thất lục bát gồm 164 câu, hay đúng ra là 164 dòng."
    },
    {
     

# Tải dữ liệu tác giả

In [15]:
current_folder = os.getcwd()
data_folder_path = os.path.abspath(os.path.join(current_folder, '..', 'Data'))

input_file_path = os.path.join(data_folder_path, 'wiki_data.json')

output_file_path = os.path.join(data_folder_path, 'wiki_sentences_2.json')


In [16]:

print(f"Đọc dữ liệu thô từ: {input_file_path}")
print(f"Lưu dữ liệu câu sạch vào: {output_file_path}")

Đọc dữ liệu thô từ: d:\STUDY DOCUMENT\4th YEAR\NLP\Vietnamese_literature\Data\wiki_data.json
Lưu dữ liệu câu sạch vào: d:\STUDY DOCUMENT\4th YEAR\NLP\Vietnamese_literature\Data\wiki_sentences_2.json


In [17]:

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

SKIP_COUNT = 57     # Bỏ qua 56 link đầu
TAKE_COUNT = 51     # Lấy 51 link tiếp theo
MAX_SENTENCES = 20  # Giới hạn 20 câu mỗi bài

print(f" File Input: {input_file_path}")
print(f" File Output (Ghi tiếp): {output_file_path}")
print(f" Bỏ qua: {SKIP_COUNT} link.")
print(f" Xử lý tiếp: {TAKE_COUNT} link.")

 File Input: d:\STUDY DOCUMENT\4th YEAR\NLP\Vietnamese_literature\Data\wiki_data.json
 File Output (Ghi tiếp): d:\STUDY DOCUMENT\4th YEAR\NLP\Vietnamese_literature\Data\wiki_sentences_2.json
 Bỏ qua: 57 link.
 Xử lý tiếp: 51 link.


In [18]:

def scrape_and_process(url, source_name):
    sentences_data = []
    try:
        decoded_url = urllib.parse.unquote(url)
        
        response = requests.get(decoded_url, headers=HEADERS, timeout=10)
        if response.status_code != 200:
            print(f" Lỗi HTTP {response.status_code}")
            return []

        soup = BeautifulSoup(response.content, 'html.parser')
        
        content_div = soup.find('div', {'id': 'mw-content-text'})
        if not content_div:
            return []

        paragraphs = content_div.find_all('p')
        
        collected_count = 0
        
        for p in paragraphs:
            if collected_count >= MAX_SENTENCES:
                break
                
            text = p.get_text()
            
            text = re.sub(r'\[\d+\]', '', text) # Xóa [1]
            text = re.sub(r'\s+', ' ', text).strip()
            
            if not text: continue
            
            sentences = re.split(r'(?<=[.!?])\s+', text)
            
            for sent in sentences:
                if collected_count >= MAX_SENTENCES:
                    break
                
                if len(sent) > 15:
                    clean_sent = sent.strip()
                    
                    sentences_data.append({
                        "ten_tac_gia": source_name, # Dùng key 'ten_tac_gia' để đồng bộ với phần 1 (hoặc 'source' tùy bạn)
                        "cau_van": clean_sent,
                    })
                    collected_count += 1
                    
        return sentences_data

    except Exception as e:
        print(f"  Lỗi Exception: {e}")
        return []

In [19]:
new_items_count = 0

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8-sig') as f:
        lines = f.readlines()
    
    valid_links = []
    for line in lines:
        parts = line.strip().split(',') if ',' in line else line.strip().split(';')
        if len(parts) >= 2 and "http" in parts[-1]:
            name = parts[0].strip().replace('"', '')
            url = parts[-1].strip().replace('"', '')
            valid_links.append((name, url))

    start_index = SKIP_COUNT
    end_index = SKIP_COUNT + TAKE_COUNT
    target_links = valid_links[start_index:end_index]
    
    print(f" Sẽ xử lý từ link thứ {start_index+1} đến {end_index}.")

    for i, (name, url) in enumerate(target_links):
        print(f"[{i+1}/{len(target_links)}] ⏳ {name}...", end=" ")
        
        data = scrape_and_process(url, name)
        
        if data:
            new_items_count += len(data)
            print(f" Thêm {len(data)} câu.")
        else:
            print(" Không lấy được câu nào.")
            
        time.sleep(0.2) 

    if new_items_count > 0:
        print(f"\n Đang lưu tổng cộng: {output_file_path}")
        with open(output_file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(" HOÀN TẤT! Dữ liệu đã được cập nhật.")
    else:
        print(" Không có dữ liệu mới nào được thêm vào.")

else:
    print(f" Không tìm thấy file input tại: {input_file_path}")

 Sẽ xử lý từ link thứ 58 đến 108.
 Không có dữ liệu mới nào được thêm vào.


In [20]:

current_folder = os.getcwd()
data_folder_path = os.path.abspath(os.path.join(current_folder, '..', 'Data'))

file_path_1 = os.path.join(data_folder_path, 'wiki_sentences_1.json') 

file_path_2 = os.path.join(data_folder_path, 'wiki_sentences_2.json')

# File đầu ra
output_file_path = os.path.join(data_folder_path, 'final_dataset.json')

print(f" File 1: {file_path_1}")
print(f" File 2: {file_path_2}")
print(f" File Output: {output_file_path}")



 File 1: d:\STUDY DOCUMENT\4th YEAR\NLP\Vietnamese_literature\Data\wiki_sentences_1.json
 File 2: d:\STUDY DOCUMENT\4th YEAR\NLP\Vietnamese_literature\Data\wiki_sentences_2.json
 File Output: d:\STUDY DOCUMENT\4th YEAR\NLP\Vietnamese_literature\Data\final_dataset.json


In [21]:
final_dataset = []
current_id = 1

# Danh sách các file cần gộp
files_to_merge = [file_path_1, file_path_2]

for file_path in files_to_merge:
    if os.path.exists(file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
            print(f"   -> Tìm thấy {len(data)} dòng dữ liệu.")
            
            count_added = 0
            for entry in data:
                text = entry.get('cau_van')
                
                if not text:
                    text = entry.get('text')

                if text:
                    # Tạo cấu trúc mới chỉ gồm ID và Text
                    new_entry = {
                        "id": current_id,
                        "text": text.strip() 
                    }
                    final_dataset.append(new_entry)
                    current_id += 1
                    count_added += 1
            
            print(f" Đã thêm {count_added} câu vào danh sách chung.")
            
        except Exception as e:
            print(f"   Lỗi khi đọc file: {e}")
    else:
        print(f"\n CẢNH BÁO: Không tìm thấy file {os.path.basename(file_path)}")




 CẢNH BÁO: Không tìm thấy file wiki_sentences_1.json

 CẢNH BÁO: Không tìm thấy file wiki_sentences_2.json


In [34]:
if final_dataset:
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(final_dataset, f, ensure_ascii=False, indent=4)
    
    # Xem thử mẫu 3 dòng đầu
    print(json.dumps(final_dataset[:3], ensure_ascii=False, indent=4))
else:
    print("\n Không có dữ liệu nào được gộp.")


 Không có dữ liệu nào được gộp.


# xóa các file không còn cần thiết 

In [ ]:

files_to_delete = [
    "wiki_data.json",                # File dữ liệu thô ban đầu
    "wiki_data2.json",           # File tách câu thử nghiệm
    "wiki_sentences_1.json",  # File kết quả phần 1
    "wiki_sentences_2.json",     # File kết quả phần 2
]

In [35]:
for file_name in files_to_delete:
    file_path = os.path.join(data_folder_path, file_name)
    
    if os.path.exists(file_path):
        try:
            os.remove(file_path) # Lệnh xóa file
            print(f"Đã xóa: {file_name}")
        except Exception as e:
            print(f" Lỗi khi xóa {file_name}: {e}")
    else:
        print(f" Không tìm thấy file (có thể đã xóa rồi): {file_name}")

print("\n HOÀN TẤT DỌN DẸP!")

Đã xóa: wiki_data.json
 Không tìm thấy file (có thể đã xóa rồi): wiki_data2.json
 Không tìm thấy file (có thể đã xóa rồi): wiki_sentences_1.json
 Không tìm thấy file (có thể đã xóa rồi): wiki_sentences_2.json

 HOÀN TẤT DỌN DẸP!
